In [4]:
from src.textclf_transformer import *
import pandas as pd
from datasets import load_dataset, concatenate_datasets
from sklearn.model_selection import train_test_split

# Tokenizacja danych

In [ ]:
ds = load_dataset("imdb")
merged = concatenate_datasets([ds["test"], ds["train"]])
df = pd.DataFrame(merged)


In [ ]:
tokenizer = WordPieceTokenizerWrapper()
tokenizer.load("src/textclf_transformer/tokenizer/BERT_original")

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=42, stratify=test_df['label'])

print(f"Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}")

Train: 40000 | Val: 5000 | Test: 5000


In [8]:
tokens_train = tokenizer.encode_pandas(train_df, 'text',
                                  max_length=512, 
                                  label_col='label')
tokens_val = tokenizer.encode_pandas(val_df, 'text',
                                max_length=512,
                                label_col='label')
tokens_test = tokenizer.encode_pandas(test_df, 'text',
                                 max_length=512,
                                 label_col='label')

In [9]:
save_dir = Path("./data/tokenized")

torch.save(tokens_train, save_dir / "imdb_train.pt")
torch.save(tokens_val, save_dir / "imdb_val.pt")
torch.save(tokens_test, save_dir / "imdb_test.pt")

# Generacja eksperymentu - finetuning

Utworzenie eksperymentu finetuning na zbiorze imdb na podstawie wcześniej wykonanego pretreningu TAPT

In [ ]:
!python experiments/generate_finetuning_experiment.py -f imdb_demo_pz -p E2_pretraining_imdb_bertsmall_favor_nb0.125

[OK] Utworzono finetuning: C:\Users\bartekb\Desktop\inzynierka\fromscratch-transformer-classifier\experiments\finetuning\imdb_demo
     config: C:\Users\bartekb\Desktop\inzynierka\fromscratch-transformer-classifier\experiments\finetuning\imdb_demo\config.yaml


# Uruchomienie finetuningu

In [ ]:
!python train.py -n imdb_demo_pz -m finetuning

^C


c:\Users\bartekb\Desktop\inzynierka\fromscratch-transformer-classifier\.venv\Lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
c:\Users\bartekb\Desktop\inzynierka\fromscratch-transformer-classifier\.venv\Lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `An

[WARN] Brakujące klucze: ['classifier.pooler.0.weight', 'classifier.pooler.0.bias', 'classifier.classifier.weight', 'classifier.classifier.bias']
[WARN] Nieoczekiwane klucze: ['mlm.transform.0.weight', 'mlm.transform.0.bias', 'mlm.transform.2.weight', 'mlm.transform.2.bias', 'mlm.decoder.weight']
One minute sleep for loading wandb!
[wandb] Initialized run 'imdb_demo' in project 'demo' (praca-inzynierska)
Epoch: 0
Epoch: 1
Epoch: 2
wandb: 
wandb: 🚀 View run imdb_demo at: https://wandb.ai/praca-inzynierska/demo/runs/tnsokald
wandb: Find logs at: experiments\finetuning\imdb_demo\wandb\run-20260107_142440-tnsokald\logs
